In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta
import time

# Functions

In [19]:
# Function to create simulation
# The parameters of the function can be change so it can semulate diferent values
def create_simulation(deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence, subType,
                      mu = 1, # Its the middle value that the distribution its going to take
                      sigma = 1, # Its the variance of the simulation
                      min = 0, # Its the min value that the simulation can take
                      max = 2): # Its the max value that the simulation can take

    # Create DataFrame
    df_dates = pd.DataFrame({'timestamp': [datetime.now()]})

    # Generate the random normal distribution
    random_num = np.random.normal(mu, sigma)

    # Add column to DataFrame
    df_dates['value'] = random_num

    df_dates['value'] = np.where(df_dates['value'] < min, min, df_dates['value'])

    df_dates['value'] = np.where(df_dates['value'] > max, max, df_dates['value'])
    df_dates['deviceName'] = deviceName
    df_dates['deviceUuid'] = deviceUuid
    df_dates['component'] = component
    df_dates['componentName'] = componentName
    df_dates['componentId'] = componentId
    df_dates['measurementType'] = measurementType
    df_dates['dataItemId'] = dataItemId
    df_dates['name'] = name
    df_dates['sequence'] = sequence
    df_dates['subType'] = subType
    df_dates['mantenance'] = 0
    
    df_dates = df_dates[['deviceName', 'deviceUuid', 'component', 'componentName', 'componentId', 'measurementType', 'dataItemId', 'timestamp', 'name', 'sequence', 'subType', 'value', 'mantenance']]

    
    return df_dates

In [1]:
# Function to load data to Mysql
def load_MySQL(table_name,new_dataframe):

    # database connection details
    db_username = 'u681559533_AMT'
    db_password = 'MVT2024VTise'
    db_host = '193.203.166.160'
    db_port = '3306'
    db_name = 'u681559533_VT_LF_DB'

    # Create the database engine
    engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

    # Load the DataFrame into MySQL table
    new_dataframe.to_sql(name=table_name, con=engine, if_exists='append', index=False)

    print("DataFrame has been loaded into the database.")

    return None

# Excecution

In [20]:
# Parameters of machine
dataItemId = 'ovenHeater1Com'
name = 'OvenHeater1CommandedTemperature'
measurementType = 'Temperature'
component = 'Enclosure'
componentName = 'Oven'
componentId = 'oven'
deviceName = 'f170'
deviceUuid = 'D12827'
sequence = 22405688
subType = 'ACTUAL'

In [22]:
# Example
create_simulation(deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence, subType)

,deviceName,deviceUuid,component,componentName,componentId,measurementType,dataItemId,timestamp,name,sequence,subType,value,mantenance
0,f170,D12827,Enclosure,Oven,oven,Temperature,ovenHeater1Com,2024-03-20 11:25:03.193940,OvenHeater1CommandedTemperature,22405688,ACTUAL,2.0,0


In [ ]:
# this is what needs to be running to keep updating data to the data base
while True:
    time.sleep(1)
    load_MySQL('test',create_simulation(deviceName, deviceUuid, component, componentName, componentId, measurementType, dataItemId, name, sequence, subType))